# 4. ionbot

In [7]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import seaborn as sns
from collections import Counter
import plotly.express as px

ModuleNotFoundError: No module named 'plotly'

In [8]:
folder = "http://genesis.ugent.be/uvpublicdata/workshop-ml-proteomics/4-Ionbot/"
dataset = "PXD001250"

In [9]:
def get_ionbot_condition(condition,folder,dataset):
    #read consition results
    ionbot_c = pd.read_csv("%s/%s%s/ionbot.first.csv"%(folder,dataset,condition))
    ionbot_c["rank"] = ["first"]*len(ionbot_c)

    tmp = pd.read_csv("%s/%s%s/ionbot.features.csv"%(folder,dataset,condition))
    ionbot_c = ionbot_c.merge(tmp,on="ionbot_match_id",how="left")

    ionbot_c["PSM"] = ionbot_c["scan"].astype(str) + ionbot_c["spectrum_file"] +  "|PSM|" +  ionbot_c["matched_peptide"]
    ionbot_c["num_matches"] = ionbot_c.groupby(["spectrum_file","scan"])["scan"].transform("count")
    return ionbot_c

def get_unique_proteins(protein_series):
    uprotein = []
    for protein in protein_series:
        uprotein.extend(protein.split("||"))
    
    return set(uprotein)

def return_search_metrics(df):

    len_psms = len(df.index)
    len_peptides = len(set(df["matched_peptide"]+"|"+df["modifications"].fillna("")))
    len_proteins = len(get_unique_proteins(df["proteins"]))

    return [len_psms, len_peptides, len_proteins]


In [10]:
# First we'll load the datasets for the open and closed modification search
ionbot = get_ionbot_condition("",folder,dataset)
ionbot_c = get_ionbot_condition("_closed",folder,dataset)
ionbot_nocorr = get_ionbot_condition("_nocorr",folder,dataset)
ionbot_nort = get_ionbot_condition("_nort",folder,dataset)
ionbot_nocorr_nort = get_ionbot_condition("_nocorr_nort",folder,dataset)


In [11]:
# Choose the level of identifications you want to plot
level = 2
level_dict = {
    0:"psms",
    1:"peptides",
    2:"proteins"
}

In [12]:
# Let's check the the number of psms for the open search versus the closed search
fig = px.bar(
    y=[return_search_metrics(ionbot)[level], return_search_metrics(ionbot_c)[level]], 
    x=[" open", "closed"],
    labels={
    "y": f"number of {level_dict[level]}",
    "x": "search condition",
    }
)
fig


NameError: name 'px' is not defined

In [ ]:
# Now let's check it for all the conditions
fig = px.bar(
    y= [
        return_search_metrics(ionbot)[level],
        return_search_metrics(ionbot_c)[level],
        return_search_metrics(ionbot_nocorr)[level],
        return_search_metrics(ionbot_nort)[level],
        return_search_metrics(ionbot_nocorr_nort)[level],
    ],
    x=["open", "closed","no correlation", "no retention time", "no correlation & no retention time"],
    labels={
    "y": f"number of {level_dict[level]}",
    "x": "search condition",
    }
)
fig

# 4.X Analysing modifications

In this tutorial we will keep the analysis of modifications very simple and short. We will look at the amount of modifications found across all PSMs.

In [13]:
def get_all_mods(df):
    mod_list = []

    for ionbot_modifications in ionbot["modifications"].fillna(""):
        mod_names = re.sub('\[.*?\]', '', ionbot_modifications)
        for mod_name in mod_names.split("|"):
            if mod_name == "":
                continue
            try:
                int(mod_name)
            except ValueError:
                mod_list.append(mod_name.lower())

    all_mods = [am for am in mod_list if all_mods != ""]
    all_mods_counter_obj = Counter(mod_list)
    return all_mods_counter_obj

all_mods_counter_obj = get_all_mods(ionbot)

NameError: name 're' is not defined

When we plot the frequency of the top 30 occurring modifications we get the following plot:

In [ ]:
plt.figure(figsize=(15,4))
plt.bar([k for k,v in all_mods_counter_obj.most_common(30)],[v for k,v in all_mods_counter_obj.most_common(30)])
plt.xticks(rotation=90)
plt.show()

## 4.X.X Questions - Analysing modifications

<ol>
  <li>What kind of modifications should be most frequently observed?</li>
  <li>Are there modifications you did not expect to see (this frequent?</li>
</ol>